https://towardsdatascience.com/keras-data-generators-and-how-to-use-them-b69129ed779c

https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c

In [1]:
!pip install tqdm
!pip install tqdm --upgrade

     |████████████████████████████████| 71kB 4.1MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [2]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D,ZeroPadding2D,Dense,Flatten,Activation,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from keras import regularizers, optimizers
from tqdm.keras import TqdmCallback
from skimage import data, io, filters,transform

In [ ]:
!unzip '/content/drive/My Drive/Máster KSchool/TFM/data/filtered_img.zip'

Image data generator

In [ ]:
data_tab = pd.read_csv('/content/drive/My Drive/Máster KSchool/TFM/data/filtered_df_with_imgNames.csv',sep=',',header= 0,usecols=['CODE','RATING'])

In [ ]:
'''data_tab = data_tab.sample(n = 10000) 
data_tab'''

In [ ]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

train_generator=datagen.flow_from_dataframe(
                        dataframe=data_tab,
                        directory="filtered_img/",
                        x_col="CODE",
                        subset="training",
                        batch_size=1,
                        seed=42,
                        shuffle=True,
                        class_mode="input",
                        target_size=(268, 182))
valid_generator=datagen.flow_from_dataframe(
                        dataframe=data_tab,
                        directory="filtered_img/",
                        x_col="CODE",
                        subset="validation",
                        batch_size=1,
                        seed=42,
                        shuffle=True,
                        class_mode="input",
                        target_size=(268,182))

In [ ]:
input_layer = tf.keras.layers.Input(shape=(268, 182, 3))

conv1 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(input_layer)#28 x 28 x 32
conv1 = tf.keras.layers.Conv2D(128, (2, 2), strides=(2,2), activation='relu', padding='same')(conv1)
conv1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1) #14 x 14 x 64
conv1 = tf.keras.layers.Conv2D(64, (2, 2), strides=(2,1), activation='relu', padding='same')(conv1)
conv1 = tf.keras.layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(conv1) #14 x 14 x 64
latent = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same')(latent)
up1 = tf.keras.layers.UpSampling2D((2,1))(conv2) 
conv2 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same')(up1)
up1 = tf.keras.layers.UpSampling2D((2,2))(conv2)

conv2 = tf.keras.layers.Conv2D(3, (3,3), activation='sigmoid', padding='same')(up1)
up1 = tf.keras.layers.UpSampling2D((2,2))(conv2)
decoded = tf.keras.layers.ZeroPadding2D((2,1))(up1)

autoencoder = Model(input_layer, decoded)

encoding_model = Model(input_layer, latent)

autoencoder.compile(loss='mean_squared_logarithmic_error', optimizer = 'adam')

autoencoder.summary()

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

autoencoder.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=30,callbacks=[TqdmCallback()], verbose=2
)

In [ ]:
data_tab_100 = data_tab.sample(n = 5)
data_tab_100

In [ ]:
test = data_tab_100.CODE.values

In [ ]:
from PIL import Image
import numpy
list =[]
for i in test:
  print(i)
  im = Image.open("filtered_img/"+i)
  np_im = numpy.array(im)/255
  list = list + [np_im] 

In [ ]:
preds = autoencoder.predict([list])
preds_medias =encoding_model.predict([list])
number= len(list)
f, ax = plt.subplots(1,number)
f.set_size_inches(16, 8)
for i in range(0,number):
    ax[i-number].imshow(list[i])

f, ax = plt.subplots(1,number)
f.set_size_inches(16, 8)
for i in range(0,number):
    ax[i-number].imshow(preds_medias[i])

f, ax = plt.subplots(1,number)
f.set_size_inches(16, 8)
for i in range(0,number):
    ax[i-number].imshow(preds[i])

plt.show()

In [3]:
def resize_with_sklearn(imgs):

  X_data_resized = [skimage.transform.resize(image, (50,32)) for image in imgs]

  return X_data_resized

In [ ]:
preds = autoencoder.predict([list])
preds_medias =encoding_model.predict([list])
number= len(list)

f, ax = plt.subplots(1,number)
f.set_size_inches(16, 8)
for i in range(0,number):
    ax[i-number].imshow(list[i])

f, ax = plt.subplots(1,number)
f.set_size_inches(16, 8)
for i in range(0,number):
    ax[i-number].imshow(resize_with_sklearn(list)[i])

plt.show()